In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data

/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
55000
10000


In [2]:
# Hyper-parameters
learningrate = 0.02
num_of_epochs=50
batchsize = 2048
num_of_iters=55000/batchsize*num_of_epochs


noutput = 10

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, noutput])

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = tf.nn.relu(conv1)
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = tf.nn.relu(conv2)
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

init_var=tf.contrib.layers.variance_scaling_initializer()
init_xavier=tf.contrib.layers.xavier_initializer()

weights = {
    'wc1': tf.get_variable("wc1", shape=[5, 5, 1, 32],initializer=init_var),
    'wc2': tf.get_variable("wc2", shape=[5, 5, 32, 64],initializer=init_var),
    'wd1': tf.get_variable("wd1", shape=[7*7*64, 1024],initializer=init_var),
    'out': tf.get_variable("out", shape=[1024,noutput],initializer=init_var)
}

biases = {
    'bc1': tf.Variable(tf.zeros([32])),
    'bc2': tf.Variable(tf.zeros([64])),
    'bd1': tf.Variable(tf.zeros([1024])),
    'out': tf.Variable(tf.zeros([noutput]))
}

In [4]:
model = conv_net(X, weights, biases, dropout=0.5)


In [5]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels=Y))
#optimizer = tf.train.AdamOptimizer(learning_rate=learningrate)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
optimizer=tf.train.MomentumOptimizer(learning_rate=learningrate, momentum=0.99)

train_min = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


tf.summary.scalar("trainingLoss", loss)
merged_summary_op = tf.summary.merge_all()
filename = "./log/mnist_project_batchsize2048" 
writer = tf.summary.FileWriter(filename, tf.get_default_graph())

init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    step=1
    while step <= num_of_iters:       
        batch_x, batch_y = mnist.train.next_batch(batchsize)  
        sess.run(train_min, feed_dict={X:batch_x, Y:batch_y})        
        losscalc, accuracycalc, merged_summary = sess.run([loss, accuracy,merged_summary_op], feed_dict={X:batch_x, Y:batch_y})
        if (step%10==1):
            print("Iteration: %d, Training Loss: %0.4f, Training Accuracy: %0.4f"%(step, losscalc, accuracycalc))
        writer.add_summary(merged_summary, step)
        step += 1
        
    test_x, test_y = mnist.test.next_batch(10000)
    losscalc, accuracycalc = sess.run([loss, accuracy], feed_dict={X:test_x, Y:test_y})
    print("Iterations: %d, Test Loss: %0.4f, Test Accuracy: %0.4f"%(step, losscalc, accuracycalc))

writer.close()               

Iteration: 1, Training Loss: 2.4870, Training Accuracy: 0.1108
Iteration: 11, Training Loss: 0.7405, Training Accuracy: 0.7598
Iteration: 21, Training Loss: 0.4817, Training Accuracy: 0.8667
Iteration: 31, Training Loss: 0.3306, Training Accuracy: 0.9028
Iteration: 41, Training Loss: 0.2252, Training Accuracy: 0.9365
Iteration: 51, Training Loss: 0.1851, Training Accuracy: 0.9482
Iteration: 61, Training Loss: 0.1482, Training Accuracy: 0.9521
Iteration: 71, Training Loss: 0.1906, Training Accuracy: 0.9531
Iteration: 81, Training Loss: 0.1351, Training Accuracy: 0.9648
Iteration: 91, Training Loss: 0.1111, Training Accuracy: 0.9678
Iteration: 101, Training Loss: 0.1104, Training Accuracy: 0.9648
Iteration: 111, Training Loss: 0.0913, Training Accuracy: 0.9756
Iteration: 121, Training Loss: 0.0838, Training Accuracy: 0.9790
Iteration: 131, Training Loss: 0.0826, Training Accuracy: 0.9766
Iteration: 141, Training Loss: 0.0480, Training Accuracy: 0.9834
Iteration: 151, Training Loss: 0.062

Iteration: 1261, Training Loss: 0.0019, Training Accuracy: 0.9995
Iteration: 1271, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 1281, Training Loss: 0.0019, Training Accuracy: 0.9990
Iteration: 1291, Training Loss: 0.0009, Training Accuracy: 0.9995
Iteration: 1301, Training Loss: 0.0009, Training Accuracy: 1.0000
Iteration: 1311, Training Loss: 0.0010, Training Accuracy: 1.0000
Iteration: 1321, Training Loss: 0.0004, Training Accuracy: 1.0000
Iteration: 1331, Training Loss: 0.0011, Training Accuracy: 0.9995
Iteration: 1341, Training Loss: 0.0010, Training Accuracy: 1.0000
Iterations: 1343, Test Loss: 0.0363, Test Accuracy: 0.9923
